# 3 Beyond A&B Testing

In Question 2 we have explored the straightforward solution to StudyPortals’ inquiry, however, there is
more to explore. In Question 3, we will design an EMM instance that extracts more information out of the StudyPortals situation.


Let's look at what would have happened if StudyPortals had designed four different variations of their webpage. For example, they might not only be interested in the two different button designs that were mentioned, but they might also be interested in knowing the effect of a new logo, a different font, or another colour, at the same time as they are testing their different buttons. As such, we would not be performing a simple A/B test, but instead we would be doing an A/B/C/D test, also known as a multivariate test.

With regular A/B testing, we had targets $t_1$, the binary column representing whether the page visitor merely viewed or also clicked, and $t_2$, the binary column representing whether the visitor was presented version A or B of the buttons. Therefore, the natural choice of EMM instance was the association model class, which allows us to determine the association between two nominal targets.

Now, with A/B/C/D testing, it still makes sense to use the association model, because we are still using two nominal targets. In this case, our targets will be $x$, the binary column representing whether the page visitor merely viewed or also clicked, and $y$, the numerical column representing whether the visitor was presented version A, B, C, or D of the webpage.

Even though we will be using integers (1, 2, 3, 4 for version A, B, C, D respectively) to code the distinct values of $y$, their values will be treated as unordered (nominal).

Having fixed the model class, we need to define an appropriate quality measure. To ensure the discovery of subgroups that represent substantial effects within the datasets, a common approach is to craft a quality measure by multiplying two components: one reflecting target deviation, and one reflecting subgroup size.

![Title](Target cross table.png)

For the quality measure component representing the target deviation, we use the target contingency table, shown in Table 1. Now, since this is no longer a 2x2 matrix, we cannot use Yule's Quality Measure here. Instead, we will have to define another appropriate quality measure.